# Clasificación por RNA de datos generados


## Caso de dos clases, una asociada a una mezcla de gaussianas multivariadas y otra con una gaussiana multivariada. **Matriz de covarianza igual**

En este código empleamos redes neuronales para clasificar datos generados.

### Generación de los datos

In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from google.colab import files

# Generación de datos aleatorios usando la mezcla de gaussianas
# Parámetros de la mezcla de 3 gaussianas
n = 5000 #Tamaño de la muestra

# definimos la matriz de medias (la fila k es el vector de medias de la k-ésima normal)
means = np.array([[-3, 0],[ 2.5, 5],[8,-0.5]])
#Matriz de covarianza
cov2=np.array([[8,0.5],[0.5,3]])

weights = [0.4, 0.2, 0.4]  # Pesos de las tres distribuciones (deben sumar 1)

#Fijamos la semilla
np.random.seed(1)

# Elegir qué componente generar para cada muestra
componentes = np.random.choice([0, 1, 2], size=n, p=weights)
# Aquí lo que hacemos es generar un vector que solo conste de 0,1 y 2's, cada uno con cierta probabilidad de aparecer (pero con elección aleatoria)
# Cada entrada j determina la distribución de la normal que debe tener el dato j

# Generamos los datos aleatorios de la clase 0
datos0 = np.array([np.random.multivariate_normal(means[i], cov2) for i in componentes])

# Separamos las coordenadas x e y para graficar
x0=datos0[:,0]
y0=datos0[:,1]

#Generación de datos de la gaussiana multivariada
# Vector de medias para x,y
mean1 = [0, -6]
# Matriz de covarianza consideramos la misma

#Generamos los datos aleatorios de la clase 1
datos1 = np.random.multivariate_normal(mean1, cov2, n)

# Separar las coordenadas x e y
x1, y1 = datos1.T  # Transponer los datos para separar las dos dimensiones



Etiquetamos nuestro datos en clases para usar estos datos fijos como modelo de entrenamiento

In [ ]:
#Clase 1
clas0=np.transpose(np.array(np.zeros(n)))
#Clase 2
clas1=np.transpose(np.array(np.ones(n)))
Clas=np.concatenate((clas0,clas1))
Clas=np.expand_dims(Clas, axis=0)  # Necesitamos un vector, por ello agregamos una dimensión al arreglo

Training_data=np.transpose(np.concatenate((datos0,datos1))) #Matriz de datos de entrenamiento

Generamos de igual manera los datos de prueba

In [ ]:
np.random.seed(3) #Fijamos una semilla para los datos de prueba
m=5000 #Número de datos de prueba
#Se repite la generación de datos aleatorios del inicio
pesos = np.random.choice([0, 1, 2], size=m, p=weights)
test0=np.array([np.random.multivariate_normal(means[i], cov2) for i in pesos])
test1 = np.random.multivariate_normal(mean1, cov2, m)
#Juntamos los datos de prueba
Test=np.transpose(np.concatenate((test0,test1)))
TestX=np.expand_dims(Test[0],axis=1)
TestY=np.expand_dims(Test[1],axis=1)

#Les asignamos las etiquetas
EtiqClas0=np.transpose(np.array(np.zeros(m)))
EtiqClas1=np.transpose(np.array(np.ones(m)))
Etiquetas=np.concatenate((EtiqClas0,EtiqClas1))
Etiquetas=np.expand_dims(Etiquetas, axis=0)


Malla sobre la cual dibujaremos nuestros datos

In [ ]:
X=np.linspace(-15,15,100) #Espaciado en x
Y=np.linspace(-12,12,100) #Espaciado en y
Mx, My = np.meshgrid(X,Y)  # Malla


Consideramos construir las probabilidades a posteriori para este caso

In [ ]:
#Determinamos la funcion de densidad de la mezcla gaussiana
def Np(x,y):
  data=np.dstack((x,y))
  dim1,dim2=x.shape
  B=np.zeros((dim1,dim2))
  for i in range(len(weights)):
    rv=stats.multivariate_normal(means[i],cov2)
    z=rv.pdf(data)
    z=weights[i]*z
    C=z.copy()
    B=B+C
  return B

def OnlyNp(x,y):
  data=np.dstack((x,y))
  dim1,dim2=x.shape
  B=np.zeros((dim1,dim2))
  rv=stats.multivariate_normal(mean1,cov2)
  z=rv.pdf(data)
  return z

#Definimos el clasificador por la regla de Bayes. En este caso estamos calculando la probabilidad P(Pi_2|x)
def Bayes(Mx,My):
  W=OnlyNp(Mx,My)
  Z=Np(Mx,My)
  P=W/(Z+W)
  return P




### Graficando puntos

En esta sección graficamos nuestros datos de entrenamiento

In [ ]:
#Determinamos la superficie generada por la probabilidad a posteriori
P=Bayes(Mx,My)
Z=Np(Mx,My)
W=OnlyNp(Mx,My)

# Graficamos los puntos
# Crear la figura y el objeto Axes3D
plt.xlim(-12,12)
plt.ylim(-12,12)
cnt=plt.contour(Mx,My,P, levels=1,colors='black')
plt.scatter(x1, y1, alpha=0.5, color='orange', label='Clase 1')
plt.scatter(x0, y0, alpha=0.5, color='teal', label='Clase 0')
plt.title('Datos de entrenamiento y regla de Bayes')
plt.clabel(cnt, cnt.levels, inline = True, fontsize = 10)
plt.xlabel('x')
plt.ylabel('y')
plt.legend(loc='upper left')
plt.savefig('mi_grafico1.png', dpi=300, bbox_inches='tight')
files.download('mi_grafico1.png')
plt.show()
#Gráfico para entender como se comportan las normales
plt.contour(Mx,My,Z, levels=50, colors='teal')
plt.contour(Mx,My,W, levels=50, colors='orange')
plt.show()



### Entrenamiento del modelo

Entrenamiento de nuestra red neuronal

In [ ]:
def shallow_NN( X, Y, lr, epochs, nh1):
  # Numero de observaciones
  m = X.shape[1]

  # Layers: input, two hidden
  n0 = X.shape[0]
  n1 = nh1
  n2 = 1

  # Condiciones iniciales
  A0 = X
  W1 = np.random.randn(n1 ,n0) * 0.01
  b1 = np.zeros((n1,1))
  #print(A0.shape, W1.shape, b1.shape)

  W2 = np.random.randn(n2 ,n1) * 0.01
  b2 = np.zeros((n2,1))

  loss = []
  for i in range(epochs):

    # Forward propagation
    Z1 = W1 @ A0 + b1
    A1 = np.tanh(Z1)

    Z2 = W2 @ A1 + b2
    A2 = ( 1.0 + np.exp(-Z2) )**(-1)

    # Backward propagation
    dZ2 = A2 - Y
    dW2 = (1.0/m) * ( dZ2 @ A1.T )
    db2 = (1.0/m) * np.sum( dZ2, axis=1, keepdims=True )


    dZ1 =  (W2.T @ dZ2) * ( 1.0 - A1**2)
    dW1 = (1.0/m) * ( dZ1 @ A0.T )
    db1 = (1.0/m) * np.sum( dZ1, axis=1, keepdims=True )


    # Actualización de los pesos
    W1 = W1 - lr*dW1
    b1 = b1 - lr*db1

    W2 = W2 - lr*dW2
    b2 = b2 - lr*db2

    loss.append( -(1.0/m)*np.sum( Y*np.log(A2) + (1.0-Y)*np.log( 1.0 - A2)) )
    #print(i, loss[-1])
  print(f'La función de costo en la iteración {epochs} es {loss[-1]}')
  return {
        'W1': W1,
        'b1': b1,
        'W2': W2,
        'b2': b2,
        'loss':loss
        }

Entrenamiento de la red

In [ ]:
nh1=1000
result=shallow_NN( Training_data, Clas, 1, 200, nh1)

Creamos la evaluación de nuestro modelo

In [ ]:
plt.plot(result['loss'], label='loss')
plt.legend()

### Predicciones de los datos de prueba
Generamos las predicciones de nuestros datos de prueba

In [ ]:
def predict( results, X ):

  A0 = X
  W1 = results['W1']
  b1 = results['b1']

  W2 = results['W2']
  b2 = results['b2']

  # Forward propagation
  Z1 = W1 @ A0 + b1
  A1 = np.tanh(Z1)

  Z2 = W2 @ A1 + b2
  A2 = ( 1.0 + np.exp(-Z2) )**(-1)

  #yhat = np.around(A2)
  return A2

Hacemos las predicciones de nuestro datos de prueba.

In [ ]:
y_test_hat = np.around(predict(result, Test))
Bayes_test_hat=np.around(Bayes(np.transpose(TestX),np.transpose(TestY)))
print(y_test_hat)
print(Bayes_test_hat)

Calculamos el porcentaje de error

In [ ]:
#Vector de errores para ambos clasificadores (si la diferencia es 0, entonces el valor se está clasificando correctamente, en caso contrario dará 1,-1)
ERROR_NN=np.abs(y_test_hat-Etiquetas)
ERROR_BAYES=np.abs(Bayes_test_hat-Etiquetas)
#Cuantificamos la cantidad de Errores cometidos entre la cantidad de datos de prueba 2m
print(f'La proporción de error de clasificación es {np.sum(ERROR_NN)/(2*m)} para la red neuronal')
print(f'La proporción de error de clasificación es {np.sum(ERROR_BAYES)/(2*m)} para la regla de Bayes')
#Calculamos

### Graficando el clasificador generado por la red neuronal
Graficamos los puntos pruebas y como fueron clasificados

In [ ]:
# Graficamos los puntos

#Primero separamos los datos como los clasifico la red neuronal
x_T0=Test[0,y_test_hat[0,:]==0]
y_T0=Test[1,y_test_hat[0,:]==0]
x_T1=Test[0,y_test_hat[0,:]==1]
y_T1=Test[1,y_test_hat[0,:]==1]

#Los datos clasificados realmente
x_R0=test0[:,0]
y_R0=test0[:,1]
x_R1=test1[:,0]
y_R1=test1[:,1]

#Generamos la regla de clasificación dada por la red neuronal
Q=np.array((Mx,My)) #2x100x100
A=np.zeros((100,100))
#Generamos la información de la superficie generada por la red sobre la malla
for i in range(100):
  for j in range(100):
    S=np.expand_dims(Q[:,i,j],axis=1)
    A[i,j]=predict(result, S)

#Cortamos para obtener la curva de nivel 0.5
cn2=plt.contour(Mx,My,A, levels=1, colors='red', linestyles= "dashed")
plt.clabel(cn2, cn2.levels, inline = True, fontsize = 8)

# Graficamos los datos clasificados según la red
plt.xlim(-12,12)
plt.ylim(-12,12)
plt.scatter(x_T0, y_T0, alpha=0.5, color='teal', label='Clase 0')
plt.scatter(x_T1, y_T1, alpha=0.5, color='orange', label='Clase 1')
plt.title(f'Datos clasificados por la red con {nh1} neuronas')

#Función de separación optima obtenida mediante la distribución de los datos
cnt=plt.contour(Mx,My,P, levels=1, colors='black')
plt.clabel(cnt, cnt.levels, inline = True, fontsize = 8)
plt.xlabel('x')
plt.ylabel('y')
plt.legend(loc='upper left')
plt.show()

#Figura de los datos aleatorios con su etiqueta real
plt.xlim(-12,12)
plt.ylim(-12,12)
plt.scatter(x_R0, y_R0, alpha=0.5, color='teal', label='Clase 0')
plt.scatter(x_R1, y_R1, alpha=0.5, color='orange', label='Clase 1')
plt.title(f'Clasificador por una red con {nh1} neuronas ocultas')

#Función de separación obtenida mediante la red
cn2=plt.contour(Mx,My,A, levels=1, colors='red', linestyles= "dashed")
plt.clabel(cnt, cnt.levels, inline = False, fontsize = 8)
#Función de separación optima obtenida mediante la distribución de los datos
cnt=plt.contour(Mx,My,P, levels=1, colors='black')
plt.clabel(cn2, cn2.levels, inline = False, fontsize = 8)
plt.annotate('Clasificador de Bayes', (-11,-5.1), fontsize=6, color='black')
plt.annotate('Clasificador de la Red', (-11,-2), fontsize=6, color='red')
plt.xlabel('x')
plt.ylabel('y')

plt.legend(loc='upper left')
plt.savefig('mi_grafico1.png', dpi=300, bbox_inches='tight')
files.download('mi_grafico1.png')
plt.show()


### Comparación de resultados cambiando el número de neuronas

En esta sección ejecutamos todos los códigos anteriores variando las neuronas en la capa oculta

In [ ]:
#Damos un vector donde cada entrada es el número de neuronas en la capa
Neurons=[1,2,3,4,5,10,20,50,100,200]

#Ejecutamos para cada cantidad de neuronas
for t in Neurons:
  result=shallow_NN( Training_data, Clas, 1, 500, nh1=t)
  #Proporción de error
  y_test_hat = np.around(predict(result, Test))
  #Vector de errores para ambos clasificadores (si la diferencia es 0, entonces el valor se está clasificando correctamente, en caso contrario dará 1,-1)
  ERROR_NN=np.abs(y_test_hat-Etiquetas)
  ERROR_BAYES=np.abs(Bayes_test_hat-Etiquetas)
  #Cuantificamos la cantidad de Errores cometidos entre la cantidad de datos de prueba 2m
  print(f'La proporción de error de clasificación es {np.sum(ERROR_NN)/(2*m)} para la red neuronal')
  print(f'La proporción de error de clasificación es {np.sum(ERROR_BAYES)/(2*m)} para la regla de Bayes')
  #Graficamos los datos reales con su etiqueta real
  plt.xlim(-12,12)
  plt.ylim(-12,12)
  plt.scatter(x_R0, y_R0, alpha=0.5, color='black', label='Clase 0')
  plt.scatter(x_R1, y_R1, alpha=0.5, color='red', label='Clase 1')
  #Por cada iteración calculamos la regla de decisión de la red neuronal
  Q=np.array((Mx,My)) #2x100x100
  A=np.zeros((100,100))
  #Graficamos la superficie
  for i in range(100):
    for j in range(100):
      S=np.expand_dims(Q[:,i,j],axis=1)
      A[i,j]=predict(result, S)

  #Cortamos para obtener la curva de nivel 0.5
  cn2=plt.contour(Mx,My,A, levels=1, colors='green', linestyles= "dashed")
  plt.clabel(cn2, cn2.levels, inline = False, fontsize = 8)
  plt.title(f'Regla de decisión con {t} neuronas')

  cnt=plt.contour(Mx,My,P, levels=1)
  plt.clabel(cnt, cnt.levels, inline = False, fontsize = 8)
  plt.xlabel('x')
  plt.ylabel('y')
  plt.annotate('Clasificador de Bayes', (-11,-5.1), fontsize=6, color='teal')
  plt.annotate('Clasificador de la Red', (-11,-2), fontsize=6, color='green')


  plt.legend(loc='upper left')
  plt.show()

## Caso de dos clases, una asociada a una mezcla de gaussianas multivariadas y otra con una gaussiana multivariada. **Matrices de covarianzas no necesariamente iguales**.

En este código empleamos redes neuronales para clasificar datos generados.

In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from google.colab import files

# Generación de datos aleatorios usando la mezcla de gaussianas
# Parámetros de la mezcla de 3 gaussianas
n = 5000 #Tamaño de la muestra

# definimos la matriz de medias (la fila k es el vector de medias de la k-ésima normal)
means = np.array([[-3, 0],[ 2.5, 5],[3,-6]])
#Matrices de covarianza
cov1=np.array([[6,0.3],[0.3,5]])
cov2=np.array([[5,0.5],[0.5,2]])
cov3=np.array([[4,0.5],[0.5,2]])

cov=np.array([[1,0],[0,2]])

# Vector de desviaciones estándar de las tres distribuciones
std_devs = np.array([cov1,cov2, cov3])
weights = [0.3,0.4,0.3]  # Pesos de las tres distribuciones (deben sumar 1)
#Fijamos la semilla
np.random.seed(1)

# Elegir qué componente generar para cada muestra
componentes = np.random.choice(range(len(weights)), size=n, p=weights)
# Aquí lo que hacemos es generar un vector que solo conste de 0,1 y 2's, cada uno con cierta probabilidad de aparecer (pero con elección aleatoria)
# Cada entrada j determina la distribución de la normal que debe tener el dato j

# Generamos los datos aleatorios de la clase 0
datos0 = np.array([np.random.multivariate_normal(means[i], std_devs[i]) for i in componentes])

# Separamos las coordenadas x e y para graficar
x0=datos0[:,0]
y0=datos0[:,1]

#Generación de datos de la gaussiana multivariada
# Vector de medias para x,y
mean1 = [7,-1]
# Matriz de covarianza consideramos la misma

#Generamos los datos aleatorios de la clase 1
datos1 = np.random.multivariate_normal(mean1, cov, n)

# Separar las coordenadas x e y
x1, y1 = datos1.T  # Transponer los datos para separar las dos dimensiones

Etiquetamos nuestros datos y generamos nuestros datos de prueba

In [ ]:
#Clase 1
clas0=np.transpose(np.array(np.zeros(n)))
#Clase 2
clas1=np.transpose(np.array(np.ones(n)))
Clas=np.concatenate((clas0,clas1))
Clas=np.expand_dims(Clas, axis=0)  # Necesitamos un vector, por ello agregamos una dimensión al arreglo

Training_data=np.transpose(np.concatenate((datos0,datos1))) #Matriz de datos de entrenamiento

np.random.seed(3) #Fijamos una semilla para los datos de prueba
m=5000 #Número de datos de prueba
#Se repite la generación de datos aleatorios del inicio
pesos = np.random.choice(range(len(weights)), size=m, p=weights)
test0=np.array([np.random.multivariate_normal(means[i], std_devs[i]) for i in pesos])
test1 = np.random.multivariate_normal(mean1, cov, m)
#Juntamos los datos de prueba
Test=np.transpose(np.concatenate((test0,test1)))
TestX=np.expand_dims(Test[0],axis=1)
TestY=np.expand_dims(Test[1],axis=1)


EtiqClas0=np.transpose(np.array(np.zeros(m)))
EtiqClas1=np.transpose(np.array(np.ones(m)))
Etiquetas=np.concatenate((EtiqClas0,EtiqClas1))
Etiquetas=np.expand_dims(Etiquetas, axis=0)

Generamos nuestra malla donde graficaremos

In [ ]:
X=np.linspace(-15,15,100) #Espaciado en x
Y=np.linspace(-12,12,100) #Espaciado en y
Mx, My = np.meshgrid(X,Y)  # Malla

Calculamos la regla de Bayes para nuestros datos

In [ ]:
#Determinamos la funcion de densidad de la mezcla gaussiana
def Np(x,y):
  data=np.dstack((x,y))
  dim1,dim2=x.shape
  B=np.zeros((dim1,dim2))
  for i in range(len(weights)):
    rv=stats.multivariate_normal(means[i],std_devs[i])
    z=rv.pdf(data)
    z=weights[i]*z
    C=z.copy()
    B=B+C
  return B

def OnlyNp(x,y):
  data=np.dstack((x,y))
  dim1,dim2=x.shape
  B=np.zeros((dim1,dim2))
  rv=stats.multivariate_normal(mean1,cov)
  z=rv.pdf(data)
  return z

#Definimos el clasificador por la regla de Bayes. En este caso estamos calculando la probabilidad P(Pi_2|x)
def Bayes(Mx,My):
  W=OnlyNp(Mx,My)
  Z=Np(Mx,My)
  P=W/(Z+W)
  return P


### Graficamos nuestros puntos

En esta sección mostramos nuestros datos de entrenamiento

In [ ]:
#Determinamos la superficie generada por la probabilidad a posteriori
P=Bayes(Mx,My)
Z=Np(Mx,My)
W=OnlyNp(Mx,My)

# Graficamos los puntos
# Crear la figura y el objeto Axes3D
plt.xlim(-12,12)
plt.ylim(-12,12)
cnt=plt.contour(X,Y,P, levels=1, colors='black')
plt.scatter(x1, y1, alpha=0.5, color='orange', label='Clase 1')
plt.scatter(x0, y0, alpha=0.5, color='teal', label='Clase 0')
plt.title('Datos de entrenamiento y regla de Bayes')
plt.clabel(cnt, cnt.levels, inline = True, fontsize = 10)
plt.xlabel('x')
plt.ylabel('y')
plt.legend(loc='upper left')
plt.savefig('mi_grafico1.png', dpi=300, bbox_inches='tight')
files.download('mi_grafico1.png')
plt.show()
#Gráfico para entender como se comportan las normales
plt.contour(Mx,My,Z, levels=50, colors='teal')
plt.contour(Mx,My,W, levels=50, colors='orange')
plt.show()

plt.contourf(Mx,My,P, levels=1)
plt.show()

### Entrenamiento del modelo

En esta sección ejecutamos el entrenamiento de la red

In [ ]:
def shallow_NN( X, Y, lr, epochs, nh1):
  # Numero de observaciones
  m = X.shape[1]

  # Layers: input, two hidden
  n0 = X.shape[0]
  n1 = nh1
  n2 = 1

  # Condiciones iniciales
  A0 = X
  W1 = np.random.randn(n1 ,n0) * 0.01
  b1 = np.zeros((n1,1))
  #print(A0.shape, W1.shape, b1.shape)

  W2 = np.random.randn(n2 ,n1) * 0.01
  b2 = np.zeros((n2,1))

  loss = []
  for i in range(epochs):

    # Forward propagation
    Z1 = W1 @ A0 + b1
    A1 = np.tanh(Z1)

    Z2 = W2 @ A1 + b2
    A2 = ( 1.0 + np.exp(-Z2) )**(-1)

    # Backward propagation
    dZ2 = A2 - Y
    dW2 = (1.0/m) * ( dZ2 @ A1.T )
    db2 = (1.0/m) * np.sum( dZ2, axis=1, keepdims=True )


    dZ1 =  (W2.T @ dZ2) * ( 1.0 - A1**2)
    dW1 = (1.0/m) * ( dZ1 @ A0.T )
    db1 = (1.0/m) * np.sum( dZ1, axis=1, keepdims=True )


    # Actualización de los pesos
    W1 = W1 - lr*dW1
    b1 = b1 - lr*db1

    W2 = W2 - lr*dW2
    b2 = b2 - lr*db2

    loss.append( -(1.0/m)*np.sum( Y*np.log(A2) + (1.0-Y)*np.log( 1.0 - A2)) )
    #print(i, loss[-1])
  print(f'La función de costo en la iteración {epochs} es {loss[-1]}')
  return {
        'W1': W1,
        'b1': b1,
        'W2': W2,
        'b2': b2,
        'loss':loss
        }

Ejecutamos el entrenamiento de nuestra red

In [ ]:
nh1=200
result=shallow_NN( Training_data, Clas, 1, 500, nh1)

Pedimos información acerca de la evolución de la función de costo

In [ ]:
plt.plot(result['loss'], label='loss')
plt.legend()

### Predicciones de los datos de prueba
Generamos las predicciones de nuestros datos de prueba

In [ ]:
def predict( results, X ):

  A0 = X
  W1 = results['W1']
  b1 = results['b1']

  W2 = results['W2']
  b2 = results['b2']

  # Forward propagation
  Z1 = W1 @ A0 + b1
  A1 = np.tanh(Z1)

  Z2 = W2 @ A1 + b2
  A2 = ( 1.0 + np.exp(-Z2) )**(-1)

  #yhat = np.around(A2)
  return A2

Hacemos las predicciones de la clase

In [ ]:
y_test_hat = np.around(predict(result, Test))
Bayes_test_hat=np.around(Bayes(np.transpose(TestX),np.transpose(TestY)))
print(y_test_hat)
print(Bayes_test_hat)

Calculamos el porcentaje de error de la red como del clasificador de la regla de Bayes. Comparamos esas proporciones.

In [ ]:
#Vector de errores para ambos clasificadores (si la diferencia es 0, entonces el valor se está clasificando correctamente, en caso contrario dará 1,-1)
ERROR_NN=np.abs(y_test_hat-Etiquetas)
ERROR_BAYES=np.abs(Bayes_test_hat-Etiquetas)
#Cuantificamos la cantidad de Errores cometidos entre la cantidad de datos de prueba 2m
print(f'La proporción de error de clasificación es {np.sum(ERROR_NN)/(2*m)} para la red neuronal')
print(f'La proporción de error de clasificación es {np.sum(ERROR_BAYES)/(2*m)} para la regla de Bayes')
#Calculamos

### Graficando el clasificador generado por la red neuronal
Graficamos los puntos pruebas y como fueron clasificados

In [ ]:
# Graficamos los puntos

#Primero separamos los datos como los clasifico la red neuronal
x_T0=Test[0,y_test_hat[0,:]==0]
y_T0=Test[1,y_test_hat[0,:]==0]
x_T1=Test[0,y_test_hat[0,:]==1]
y_T1=Test[1,y_test_hat[0,:]==1]

#Los datos clasificados realmente
x_R0=test0[:,0]
y_R0=test0[:,1]
x_R1=test1[:,0]
y_R1=test1[:,1]

#Generamos la regla de clasificación dada por la red neuronal
Q=np.array((Mx,My)) #2x100x100
A=np.zeros((100,100))
#Generamos la información de la superficie generada por la red sobre la malla
for i in range(100):
  for j in range(100):
    S=np.expand_dims(Q[:,i,j],axis=1)
    A[i,j]=predict(result, S)

#Cortamos para obtener la curva de nivel 0.5
cn2=plt.contour(Mx,My,A, levels=1, colors='red', linestyles= "dashed")
plt.clabel(cn2, cn2.levels, inline = True, fontsize = 8)

# Graficamos los datos clasificados según la red
plt.xlim(-12,12)
plt.ylim(-12,12)
plt.scatter(x_T0, y_T0, alpha=0.5, color='teal', label='Clase 0')
plt.scatter(x_T1, y_T1, alpha=0.5, color='orange', label='Clase 1')
plt.title(f'Datos clasificados por la red con {nh1} neuronas')

#Función de separación optima obtenida mediante la distribución de los datos
cnt=plt.contour(Mx,My,P, levels=1, colors='black')
plt.clabel(cnt, cnt.levels, inline = True, fontsize = 8)
plt.xlabel('x')
plt.ylabel('y')
plt.legend(loc='upper left')
plt.show()

#Figura de los datos aleatorios con su etiqueta real
plt.xlim(-12,12)
plt.ylim(-12,12)
plt.scatter(x_R0, y_R0, alpha=0.5, color='teal', label='Clase 0')
plt.scatter(x_R1, y_R1, alpha=0.5, color='orange', label='Clase 1')
plt.title(f'Clasificador por una red con {nh1} neuronas ocultas')

#Función de separación obtenida mediante la red
cn2=plt.contour(Mx,My,A, levels=1, colors='red', linestyles= "dashed")
plt.clabel(cn2, cn2.levels, inline = True, fontsize = 10)
#Función de separación optima obtenida mediante la distribución de los datos
cnt=plt.contour(Mx,My,P, levels=1, colors='black')
plt.clabel(cnt, cnt.levels, inline = True, fontsize = 10)
plt.annotate('Clasificador de Bayes', (5,-6), fontsize=8, color='black')
plt.annotate('Clasificador de la Red', (5,5), fontsize=8, color='red')
plt.xlabel('x')
plt.ylabel('y')
plt.legend(loc='upper left')
plt.savefig('mi_grafico1.png', dpi=300, bbox_inches='tight')
files.download('mi_grafico1.png')
plt.show()

### Comparación de resultados cambiando el número de neuronas

En esta sección ejecutamos todos los códigos anteriores variando las neuronas en la capa oculta

In [ ]:
#Damos un vector donde cada entrada es el número de neuronas en la capa
Neurons=[1,2,3,4,5,10,20,50,100,200,1000]

#Ejecutamos para cada cantidad de neuronas
for t in Neurons:
  result=shallow_NN( Training_data, Clas, 1, 200, nh1=t)
  #Proporción de error
  y_test_hat = np.around(predict(result, Test))
  #Vector de errores para ambos clasificadores (si la diferencia es 0, entonces el valor se está clasificando correctamente, en caso contrario dará 1,-1)
  ERROR_NN=np.abs(y_test_hat-Etiquetas)
  ERROR_BAYES=np.abs(Bayes_test_hat-Etiquetas)
  #Cuantificamos la cantidad de Errores cometidos entre la cantidad de datos de prueba 2m
  print(f'La proporción de error de clasificación es {np.sum(ERROR_NN)/(2*m)} para la red neuronal')
  print(f'La proporción de error de clasificación es {np.sum(ERROR_BAYES)/(2*m)} para la regla de Bayes')

  #Graficamos los datos reales con su etiqueta real
  plt.xlim(-12,12)
  plt.ylim(-12,12)
  plt.scatter(x_R0, y_R0, alpha=0.5, color='black', label='Clase 0')
  plt.scatter(x_R1, y_R1, alpha=0.5, color='red', label='Clase 1')
  plt.title('Datos con clasificación real')
  #Por cada iteración determinamos el porcentaje de error de clasificación
  y_test_hat = np.around(predict(result, Test))
  ERROR=np.abs(y_test_hat-Etiquetas) #Vector de errores (si la diferencia es 0, entonces el valor se está clasificando correctamente, en caso contrario dará 1,-1)
  print(f'La proporción de error de clasificación es {np.sum(ERROR)/(2*m)}') #Cuantificamos la cantidad de Errores cometidos entre la cantidad de datos de prueba 2m

  #Por cada iteración calculamos la regla de decisión de la red neuronal
  Q=np.array((Mx,My)) #2x100x100
  A=np.zeros((100,100))
  #Graficamos la superficie
  for i in range(100):
    for j in range(100):
      S=np.expand_dims(Q[:,i,j],axis=1)
      A[i,j]=predict(result, S)

  #Cortamos para obtener la curva de nivel 0.5
  cn2=plt.contour(Mx,My,A, levels=1, colors='green', linestyles= "dashed")
  plt.clabel(cn2, cn2.levels, inline = False, fontsize = 8)
  plt.title(f'Regla de decisión con {t} neuronas')

  cnt=plt.contour(Mx,My,P, levels=1)
  plt.clabel(cnt, cnt.levels, inline = False, fontsize = 8)
  plt.xlabel('x')
  plt.ylabel('y')

  plt.legend(loc='upper left')
  plt.show()

  plt.contourf(Mx,My,P, levels=1, alpha=0.4)
  plt.contour(Mx,My,A, levels=1, colors='black', linestyles= "dashed")
  plt.show()